In [ ]:
# install Pytorch: https://pytorch.org/

# import necessary modules
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import os

current_dir = os.getcwd()
print(current_dir)
# Use our paintings data (Monet & Picasso images)
train_images_folder = os.path.abspath(os.path.join(current_dir, '..', 'presplit_data/train/'))
validation_images_folder = os.path.abspath(os.path.join(current_dir, '..', 'presplit_data/validation/'))
#Create our train and validation dataset 
train_dataset = torchvision.datasets.ImageFolder(root=train_images_folder,  transform=transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Resize((256,256))]))
validation_dataset = torchvision.datasets.ImageFolder(root=validation_images_folder, transform=transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Resize((256,256))]))
from torch.utils.data import DataLoader

/Users/margaretsnape/Documents/Coursera/Imperial/Unstructured Data Analysis/UDA_HW4/code


In [8]:
# divide the dataset into batches
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset,
                                          batch_size=100, 
                                          shuffle=False)

In [18]:
# Convolutional neural network (two convolutional layers)
class UDA_CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(UDA_CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(
                    #in_channels = 1,
                    in_channels = 3,
                    out_channels = 16,
                    kernel_size=5,
                    stride=1,
                    padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        #self.fc = nn.Linear(7*7*32, num_classes)
        self.fc = nn.Linear(2*256*256, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out
      
model = UDA_CNN()

In [ ]:
# Loss function
criterion = nn.CrossEntropyLoss()
# Optimizer to update network parameters
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model with our images and labels
total_step = len(train_loader)
for epoch in range(2):
    for i, (images, labels) in enumerate(train_loader):
        images = images
        labels = labels
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, 5, i+1, total_step, loss.item()))

In [22]:
# Test the model
model.eval()  # turn off batchnorm layer
with torch.no_grad(): # turn off gradient computation
    correct = 0
    total = 0
    for images, labels in validation_loader:
        images = images
        labels = labels
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the ',total ,' test images: {} %'.format(100 * correct / total))
    
# Save the model checkpoint
torch.save(model.state_dict(), 'model.uda')

Test Accuracy of the model on the  418  test images: 81.10047846889952 %
